# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Optimise for single wind speed for better initial estimate
    - Power not LCOE
    - Different values for below/at rated power
- Increase fidelity
    - Assess difference in speed of calculation and optimal values
- Loop over wind direction
    - Parallelise using PQDM
- Assess results
    - Sensitivity to wind direction/speed
    - Variation in sensitivity between turbines
    - Increase number of turbines

## Setup

In [1]:
# import libraries
import logging

import numpy as np
from scipy import optimize

import utils

In [2]:
# define constants
yaw_scale = 30

## Baseline values

In [3]:
# run baseline simulation
sim_res_base = utils.run_sim()

In [4]:
# ensure probabilities (wind direction and speed) total 1
Sector_frequency = sim_res_base.Sector_frequency
if not np.isclose(Sector_frequency.sum(), 1):
    logging.warning(
        f"Sector frequency renormalised as total probability was {Sector_frequency.sum().values}"
    )
    Sector_frequency = Sector_frequency / Sector_frequency.sum()
P = sim_res_base.P
if not np.isclose(P.sum(), 1):
    logging.warning(f"P renormalised as total probability was {P.sum().values}")
    P = P / P.sum()

In [14]:
# calculate baseline metrics
aep_base, lcoe_base, cap_fac_base = utils.calc_metrics(
    sim_res=sim_res_base,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)
(
    lcoe_direction_base,
    cap_fac_direction_base,
    lcoe_overall_base,
    cap_fac_overall_base,
) = utils.aggregate_metrics(
    aep=aep_base,
    lcoe=lcoe_base,
    cap_fac=cap_fac_base,
    Sector_frequency=Sector_frequency,
)

AEP [GWh]: 79.174
LCoE [USD/MWh]: 44.477
Capacity factor [%]: 50.177


## Optimise across for a single wind direction

In [6]:
# define constants
wd = 270
yaw_shape = (len(sim_res_base.wt), 1, len(sim_res_base.ws))

In [7]:
# define objective function
def obj_single(yaw_norm):
    sim_res = utils.run_sim(yaw=yaw_norm.reshape(yaw_shape) * yaw_scale, wd=wd)
    aep, lcoe, _ = utils.calc_metrics(
        sim_res=sim_res,
        sim_res_base=sim_res_base,
        Sector_frequency=Sector_frequency,
        P=P,
    )
    _, _, lcoe_overall, _ = utils.aggregate_metrics(
        aep=aep, lcoe=lcoe, Sector_frequency=Sector_frequency
    )
    obj = (lcoe_overall / lcoe_direction_base.sel(wd=wd)).values.tolist()
    return obj

In [8]:
# check objective function correct for baseline case
assert np.isclose(obj_single(np.zeros(yaw_shape)), 1)

In [9]:
# define optimisation callback
def _callback(intermediate_result):
    global n
    if n % 5 == 0:
        print(f"Iteration {n:3.0f}: obj = {intermediate_result.fun:.5f}")
    n += 1

In [10]:
# run optimisation with iteration counter display
n = 0
res = optimize.minimize(
    fun=obj_single, x0=np.zeros(yaw_shape).ravel() + 0.1 / yaw_scale, callback=_callback
)
res

Iteration   0: obj = 0.99833
Iteration   5: obj = 0.99352
Iteration  10: obj = 0.99339
Iteration  15: obj = 0.99315
Iteration  20: obj = 0.99306
Iteration  25: obj = 0.99300
Iteration  30: obj = 0.99292
Iteration  35: obj = 0.99284
Iteration  40: obj = 0.99279
Iteration  45: obj = 0.99270
Iteration  50: obj = 0.99245
Iteration  55: obj = 0.99235
Iteration  60: obj = 0.99233
Iteration  65: obj = 0.99229
Iteration  70: obj = 0.99228
Iteration  75: obj = 0.99224
Iteration  80: obj = 0.99224
Iteration  85: obj = 0.99222
Iteration  90: obj = 0.99222
Iteration  95: obj = 0.99221
Iteration 100: obj = 0.99220
Iteration 105: obj = 0.99219
Iteration 110: obj = 0.99219
Iteration 115: obj = 0.99218
Iteration 120: obj = 0.99218
Iteration 125: obj = 0.99218
Iteration 130: obj = 0.99218
Iteration 135: obj = 0.99218
Iteration 140: obj = 0.99218
Iteration 145: obj = 0.99218
Iteration 150: obj = 0.99217
Iteration 155: obj = 0.99217
Iteration 160: obj = 0.99217
Iteration 165: obj = 0.99217
Iteration 170:

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.9921638155738236
        x: [ 3.333e-03  3.333e-03 ...  3.333e-03  3.333e-03]
      nit: 227
      jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
            ...
            [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
            [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
     nfev: 70000
     njev: 250

In [19]:
# rerun simulation for optimum
sim_res_opt = utils.run_sim(yaw=res.x.reshape(yaw_shape) * yaw_scale, wd=wd)
aep_opt, lcoe_opt, cap_fac_opt = utils.calc_metrics(
    sim_res=sim_res_opt,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
)
_, _, lcoe_overall_opt, cap_fac_overall_opt = utils.aggregate_metrics(
    aep=aep_opt, lcoe=lcoe_opt, cap_fac=cap_fac_opt, Sector_frequency=Sector_frequency
)

In [20]:
# display comaprison of optimum to baseline
print(f"Baseline LCoE [USD/MWh]      : {lcoe_direction_base.sel(wd=wd).values:.3f}")
print(f"Optimum LCoE [USD/MWh]       : {lcoe_overall_opt.values:.3f}")
print(
    f"Baseline Capacity Factor [%] : {100*cap_fac_direction_base.sel(wd=wd).values:.3f}"
)
print(f"Optimum Capacity Factor [%]  : {100*cap_fac_overall_opt.values:.3f}")

Baseline LCoE [USD/MWh]      : 43.252
Optimum LCoE [USD/MWh]       : 42.913
Baseline Capacity Factor [%] : 51.598
Optimum Capacity Factor [%]  : 52.006
